In [1]:
# =============================================================================
# ENVIRONMENT CHECK - Run this cell first!
# =============================================================================
# This cell verifies that the DEGIS environment is properly set up
# Make sure you've run ./setup_server_fixed.sh first!

import sys
import os

# Check if we're in the right environment
if 'degis-env' in sys.executable:
    print("DEGIS environment is active")
    print(f"Python: {sys.executable}")
else:
    print("Warning: DEGIS environment not detected")
    print("Please run: ./setup_server_fixed.sh")
    print("Then activate: source degis-env/bin/activate")

# Check if DEGIS package is available
try:
    import degis
    print("DEGIS package is available")
except ImportError:
    print("DEGIS package not found")
    print("Please run: ./setup_server_fixed.sh")

print("\nReady to start image generation!")

DEGIS environment is active
Python: /data/degis/degis-env/bin/python
DEGIS package is available

Ready to start image generation!


In [2]:
import geomloss
import numpy as np
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from PIL import Image
from torchvision import transforms
from IPython.display import display
import os
import glob

# Import DEGIS package components
from degis import IPAdapterXLGenerator, load_trained_color_head, get_color_embedding
from degis.shared.utils import plot_color_palette, display_images_grid, display_comparison_grid
from degis.shared.image_features.color_histograms import compute_color_histogram, compute_lab_histogram
from degis.shared.utils.image_utils import create_control_edge_pil
from degis.data.dataset import UnifiedImageDataset

# Import IP-Adapter XL with DEGIS patches
import ip_adapter_patch  # This applies the DEGIS patches
from diffusers import ControlNetModel, StableDiffusionXLControlNetPipeline

print("All imports successful!")
print("Ready for high-quality image generation with IP-Adapter XL!")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

All imports successful!
Ready for high-quality image generation with IP-Adapter XL!
Using device: cuda


In [3]:
# csv_path = "/data/thesis/laion_5m_manifest.csv"
# embeddings_path = "/data/thesis/models/hf_xl_laion_5m_embeddings.npy" # || YOUR_CUSTOM_PATH
# colour_path = "/data/thesis/data/laion_5m_color_histograms_lab_514.npy" # options: COLOR_HIST_PATH_LAB_514 || COLOR_HIST_PATH_RGB || YOUR_CUSTOM_PATH
# colour_head_checkpoint_path = "/data/degis/evaluation_runs/laion_5m_xl_lab514_tk20_b4096/best_color_head_tmp.pth" # || YOUR_CUSTOM_PATH
# precomputed_adimagenet_edge_maps_path = "/data/thesis/data/adimagenet_edge_maps.npy" # || YOUR_CUSTOM_PATH

# image_encoder_path = "laion/CLIP-ViT-bigG-14-laion2B-39B-b160k"
# controlnet_id = "diffusers/controlnet-canny-sdxl-1.0"

In [11]:
coco_csv_path = "/data/thesis/coco_manifest.csv"
coco_df = pd.read_csv(coco_csv_path)
print(coco_df.shape)
coco_df.head()

(616767, 5)


,split,image_id,file_name,local_path,caption
0,train,203564,000000203564.jpg,/data/thesis/coco/images/train2017/00000020356...,A bicycle replica with a clock as the front wh...
1,train,322141,000000322141.jpg,/data/thesis/coco/images/train2017/00000032214...,A room with blue walls and a white sink and door.
2,train,16977,000000016977.jpg,/data/thesis/coco/images/train2017/00000001697...,A car that seems to be parked illegally behind...
3,train,106140,000000106140.jpg,/data/thesis/coco/images/train2017/00000010614...,A large passenger airplane flying through the ...
4,train,106140,000000106140.jpg,/data/thesis/coco/images/train2017/00000010614...,There is a GOL plane taking off in a partly cl...


In [13]:
laion_csv_path = "/data/thesis/laion_5m_manifest.csv"
laion_df = pd.read_csv(laion_csv_path)
print(laion_df.shape)
laion_df.head()

(3336240, 5)


,id,url,caption,aesthetic,local_path
0,6744,https://t0.gstatic.com/images?q=tbn:ANd9GcQM-D...,Wrought Iron King Headboard And Footboard by B...,7.297200,/data/thesis/laion_5m_images/0006744.jpg
1,5801,https://i.dailymail.co.uk/1s/2020/03/24/16/263...,Kayley and Ryan said they were determined to '...,7.451863,/data/thesis/laion_5m_images/0005801.jpg
2,8959,https://cdn.shopify.com/s/files/1/2068/6307/pr...,Pink Minnie Mouse suspender dress,7.530277,/data/thesis/laion_5m_images/0008959.jpg
3,9283,http://t0.gstatic.com/images?q=tbn:ANd9GcSxmV1...,Xmas Home Decorating Ideas by Best 25 Christma...,7.742550,/data/thesis/laion_5m_images/0009283.jpg
4,2750,https://cdn.smokymountains.com/vacation-rental...,Photo of a Gatlinburg Cabin named The Blue Spr...,7.035913,/data/thesis/laion_5m_images/0002750.jpg


In [ ]:
adimagenet_csv_path = "/data/thesis/adimagenet_manifest.csv"
adimagenet_df = pd.read_csv(adimagenet_csv_path)
print(adimagenet_df.shape)
adimagenet_df.head()

(2080, 6)


,file_path,file_name,text,dimensions,width,height
0,"/data/thesis/AdImageNet/images/(300, 250)/ad_0...",ad_000001.jpg,$3\nSTULZ\nDifferential for 2nd Shift\nManufac...,"(300, 250)",300,250
1,"/data/thesis/AdImageNet/images/(300, 250)/ad_0...",ad_000009.jpg,VULTURE\ninto\nwith\nSam Sanders\nApple Podcasts,"(300, 250)",300,250
2,"/data/thesis/AdImageNet/images/(300, 250)/ad_0...",ad_000017.jpg,smart\ncare\nO\ndesign\nbuild\n& install\nrepa...,"(300, 250)",300,250
3,"/data/thesis/AdImageNet/images/(300, 600)/ad_0...",ad_000020.jpg,"TREE\nSANTOR\nMatch On!\nThe Showstopper,\nLuc...","(300, 600)",300,600
4,"/data/thesis/AdImageNet/images/(300, 250)/ad_0...",ad_000021.jpg,Local experts connecting\ncustomers to YOUR bu...,"(300, 250)",300,250


: 